## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

## Reading-In and Wrangling Data

In [ ]:
df_spy = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "spy")
df_agg = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "agg")
df_hyg = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "hyg")
df_tlt = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "tlt")
df_gld = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "gld")
df_buffer_010 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mqu1bslq")
df_buffer_020 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mquslblr")
df_buffer_100 = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "mqu1pplr")
df_sv_hedged_income = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_income")
df_sv_hedged_balanced = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_balanced")
df_sv_hedged_enhanced_growth = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_hedged_enhanced_growth")
df_sv_equity_buffer = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_equity_buffer")
df_sv_equity_buffer_growth = pd.read_excel("..\\data\\bufr_bufd_mquslblr.xlsx", "sv_equity_buffer_growth")
df_spy_corrections = pd.read_csv("..//data/spy_corrections.csv")
df_spy_corrections["start_date"] = pd.to_datetime(df_spy_corrections["start_date"])
df_spy_corrections["end_date"] = pd.to_datetime(df_spy_corrections["end_date"])

In [ ]:
df_px = (
    df_buffer_100
        .merge(df_buffer_010, how="left", on="date")
        .merge(df_buffer_020, how="left", on="date")
        .merge(df_spy, how="left", on="date")
        .merge(df_agg, how="left", on="date")
        .merge(df_hyg, how="left", on="date")
        .merge(df_tlt, how="left", on="date")
        .merge(df_gld, how="left", on="date")
        .merge(df_sv_hedged_income, how="left", on="date")
        .merge(df_sv_hedged_balanced, how="left", on="date")
        .merge(df_sv_hedged_enhanced_growth, how="left", on="date")
        .merge(df_sv_equity_buffer, how="left", on="date")
        .merge(df_sv_equity_buffer_growth, how="left", on="date")
)
df_px.rename(columns={"mqu1pplr":"buffer_100", "mquslblr":"buffer_020", "mqu1bslq":"buffer_010"}, inplace=True)
df_px

,date,buffer_100,buffer_010,buffer_020,spy,agg,hyg,tlt,gld,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,2006-12-29,1000.0000,1110.43,1099.0969,100.739937,57.753967,NaN,51.180542,63.209999,NaN,NaN,NaN,NaN,NaN
1,2007-01-03,999.8752,1109.92,1097.9915,100.562088,57.875610,NaN,51.545147,62.279999,NaN,NaN,NaN,NaN,NaN
2,2007-01-04,1000.9686,1111.59,1099.8248,100.775497,57.997261,NaN,51.857674,61.650002,NaN,NaN,NaN,NaN,NaN
3,2007-01-05,998.3562,1107.75,1094.9608,99.971687,57.956741,NaN,51.631977,60.169998,NaN,NaN,NaN,NaN,NaN
4,2007-01-08,1000.3551,1110.56,1097.9422,100.434090,57.991474,NaN,51.724541,60.480000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4553,2025-02-05,1540.2790,3921.65,2933.1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4554,2025-02-06,1541.3184,3929.57,2935.2570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4555,2025-02-07,1538.6975,3911.23,2927.5914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4556,2025-02-10,1541.6280,3926.20,2936.4200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#df_spy_corrections.info()

## Helper Functions

In [ ]:
def max_period_drawdown(asset, date_start, date_end, df_ret):
    col_name = f"equity_{asset}"
    df_temp = df_ret.query("@date_start <= date <= @date_end")[["date", col_name]].copy()
    df_temp["drawdown"] = (df_temp[col_name] / df_temp[col_name].cummax()) - 1
    return df_temp["drawdown"].min()

In [ ]:
def rebalanced_portfolio(df_ret, assets, weights, frequency_rebalance="quarterly"):
    frequency_rebalance = frequency_rebalance
    df = df_ret[["date"]].copy()

    # determining rebalance dates
    if frequency_rebalance == "annual":
        df_ret["year"] = df_ret["date"].dt.year
        df["year"] = df["date"].dt.year
        df_date_rebalance = df.groupby(["year"])[["date"]].max().reset_index()
        df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
        df_ret = df_ret.merge(df_date_rebalance, how="left", on=["year"])
    elif frequency_rebalance == "semiannual":
        df_ret["year"] = df_ret["date"].dt.year
        df_ret["month"] = df_ret["date"].dt.month
        df_ret["half"] = np.where(df_ret["month"] <= 6, 1, 2)
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
        df["half"] = np.where(df["month"] <= 6, 1, 2)
        df_date_rebalance = df.groupby(["year","half"])[["date"]].max().reset_index()
        df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
        df_ret = df_ret.merge(df_date_rebalance, how="left", on=["year", "half"])
    elif frequency_rebalance == "quarterly":
        df_ret["year"] = df_ret["date"].dt.year
        df_ret["quarter"] = df_ret["date"].dt.quarter
        df["year"] = df["date"].dt.year
        df["quarter"] = df["date"].dt.quarter
        df_date_rebalance = df.groupby(["year","quarter"])[["date"]].max().reset_index()
        df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
        df_ret = df_ret.merge(df_date_rebalance, how="left", on=["year", "quarter"])
    elif frequency_rebalance == "monthly":
        df_ret["year"] = df_ret["date"].dt.year
        df_ret["month"] = df_ret["date"].dt.month
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
        df_date_rebalance = df.groupby(["year","month"])[["date"]].max().reset_index()
        df_date_rebalance.rename(columns={"date":"date_rebalance"}, inplace=True)
        df_ret = df_ret.merge(df_date_rebalance, how="left", on=["year", "month"])
    elif frequency_rebalance == "daily":
        df_ret["date_rebalance"] = df_ret["date"]
        # display(df_ret)
        # print(df_ret.info())

    # initializing values for iteration through df_ret
    before_rebal = {}
    lst_total_value = []
    total_value = 0
    for ix_asset in assets:
        before_rebal[ix_asset] = [portfolio[ix_asset]]
        total_value += before_rebal[ix_asset][-1]
    lst_total_value.append(total_value)
    after_rebal = {}
    for ix_asset in assets:
        after_rebal[ix_asset] = [portfolio[ix_asset]]

    # iterating through df_ret to calcuate portfolio values
    for _ , row in df_ret[1:].iterrows():
        # calculating EOD value of each asset allocation
        for ix_asset, _ in before_rebal.items():
            before_rebal[ix_asset].append(after_rebal[ix_asset][-1] * (1 + row["ret_"+ ix_asset]))
        
        # calculating total value before rebalance
        total_value = 0
        for ix_asset, _ in before_rebal.items():
            total_value += before_rebal[ix_asset][-1]
        lst_total_value.append(total_value)    
    
        # rebalancing if needed
        if row["date"] == row["date_rebalance"]:
            for ix_asset, _ in after_rebal.items():
                after_rebal[ix_asset].append(total_value * portfolio[ix_asset])
        else:
            for ix_asset, _ in after_rebal.items():
                after_rebal[ix_asset].append(before_rebal[ix_asset][-1])

    # adding columns to df_ret
    df_before_rebal = pd.DataFrame(before_rebal)
    for ix_asset, _ in before_rebal.items():
        df_before_rebal.rename(columns={ix_asset: "before_rebal_" + ix_asset}, inplace=True)
    df_before_rebal
    df_after_rebal = pd.DataFrame(after_rebal)
    for ix_asset, _ in after_rebal.items():
        df_after_rebal.rename(columns={ix_asset: "after_rebal_" + ix_asset}, inplace=True)
    df_after_rebal
    df_total_value = pd.DataFrame({"portfolio_total_value":lst_total_value})
    df_ret = pd.concat([df_ret, df_before_rebal, df_total_value, df_after_rebal], axis=1)
    df_ret["ret_portfolio"] = df_ret["portfolio_total_value"].pct_change()
    df_ret.fillna(0, inplace=True)
    # display(df_ret)
    # print(df_ret.info())
    
    return df_ret

## Single Backtest

In [ ]:
date_start = '2022-06-30'
date_end = '2024-12-31'

portfolio = {
    "spy":0.5,
    "agg":0.5,
}

# portfolio = {
#     "spy": 1/3,
#     "agg": 1/3,
#     "sv_equity_buffer": 1/3,
# }

In [ ]:
def backtester(portfolio=portfolio, date_start=date_start, date_end=date_end, frequency_rebalance="quarterly", df_px=df_px, df_spy_corrections=df_spy_corrections):
    # filtering for only the dates that we need
    df_px = df_px.query("@date_start <= date & date <= @date_end").copy()
    df_spy_corrections = df_spy_corrections.query("@date_start <= start_date & end_date <= @date_end").copy()

    # isolating weights and assets from portfolio
    assets = []
    weights = []
    for asset, weight in portfolio.items():
        assets.append(asset)
        weights.append(weight)

    # grabbing only the columns needed from df_px
    cols = ["date"] + assets
    df_ret = df_px[cols].reset_index(drop=True).copy()

    # calculating component asset daily returns
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        df_ret[ret_col_name] = df_ret[ix_asset].pct_change()
    df_ret.fillna(0, inplace=True)

    # calculating portfolio daily returns
    # cols = []
    # for ix_asset in assets:
    #     cols.append("ret_" + ix_asset)
    # df_ret["ret_portfolio"] =  np.sum(np.array(df_ret[cols]) * weights, axis=1)
    df_ret = rebalanced_portfolio(df_ret, assets, weights, frequency_rebalance)

    # calculating equity curve for components and portfolio
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        equity_col_name = "equity_" + ix_asset
        df_ret[equity_col_name] = (1 + df_ret[ret_col_name]).cumprod()
    df_ret["equity_portfolio"] = (1 + df_ret["ret_portfolio"]).cumprod()

    # calculating drawdowns for components and portfolio
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        drawdown_col_name = "drawdown_" + ix_asset
        df_ret[drawdown_col_name] = (df_ret[equity_col_name] / df_ret[equity_col_name].cummax()) - 1
    df_ret["drawdown_portfolio"] = (df_ret["equity_portfolio"] / df_ret["equity_portfolio"].cummax()) - 1

    # calculating cumulative returns
    cumulative_returns = {}
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        cumulative_returns[ix_asset] = (df_ret[equity_col_name].iloc[-1] - 1)
    cumulative_returns["portfolio"] = df_ret["equity_portfolio"].iloc[-1] - 1

    # calculating annual returns
    annual_returns = {}
    for ix_asset in assets:
        equity_col_name = "equity_" + ix_asset
        annual_returns[ix_asset] = (df_ret[equity_col_name].iloc[-1] ** (252/(len(df_ret) - 1)) - 1)
    annual_returns["portfolio"] = df_ret["equity_portfolio"].iloc[-1] ** (252/(len(df_ret) - 1)) - 1

    # calculating volatilties
    volatility = {}
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        volatility[ix_asset] = df_ret[ret_col_name][1:].std() * np.sqrt(252)
    volatility["portfolio"] = df_ret["ret_portfolio"][1:].std() * np.sqrt(252)

    # calculating sharpe-ratio
    sharpe = {}
    for ix_asset in assets:
        ret_col_name = "ret_" + ix_asset
        sharpe[ix_asset] = (df_ret[ret_col_name][1:].mean() / df_ret[ret_col_name][1:].std()) * np.sqrt(252)
    sharpe["portfolio"] = (df_ret["ret_portfolio"][1:].mean() / df_ret["ret_portfolio"][1:].std()) * np.sqrt(252)

    # calendar year performance
    df_portfolio = df_ret[["date", "ret_portfolio"]].copy()
    #df_portfolio["date"] = pd.to_datetime(df_portfolio["date"])
    df_portfolio["year"] = df_portfolio["date"].dt.year
    df_annual_performance = df_portfolio.groupby(["year"])[["ret_portfolio"]].agg(lambda x: np.prod(1 + x) - 1).reset_index()
    
    # maximum drawdown
    drawdown_max = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_max[ix_asset] = df_ret[drawdown_col_name].min()
    drawdown_max["portfolio"] =  df_ret["drawdown_portfolio"].min()

    # gfc drawdown
    start = "2007-10-09"
    end = "2009-03-09"
    drawdown_gfc = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_gfc[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_gfc["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # pandemic drawdown
    start = "2020-02-19"
    end = "2020-03-23"
    drawdown_pandemic = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_pandemic[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_pandemic["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # 2022 drawdown
    start = "2022-01-03"
    end = "2022-10-12"
    drawdown_2022 = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_2022[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_2022["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # summer 2024 drawdown
    start = "2024-08-01"
    end = "2024-08-05"
    drawdown_summer_2024 = {}
    for ix_asset in assets:
        drawdown_col_name = "drawdown_" + ix_asset
        drawdown_summer_2024[ix_asset] = max_period_drawdown(asset=ix_asset, date_start=start, date_end=end, df_ret=df_ret)
    drawdown_summer_2024["portfolio"] = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)

    # for all market corrections
    drawdowns_portfolio = []
    for ix in df_spy_corrections.index:
        start_date = df_spy_corrections.at[ix, "start_date"]
        end_date = df_spy_corrections.at[ix, "end_date"]
        drawdown_portfolio = max_period_drawdown(asset="portfolio", date_start=start, date_end=end, df_ret=df_ret)
        drawdowns_portfolio.append(drawdown_portfolio)
    df_corrections = df_spy_corrections.copy()
    df_corrections["drawdown_portfolio"] = drawdowns_portfolio

    result = {
        "portfolio":portfolio,
        "cumulative_return":cumulative_returns,
        "annual_return":annual_returns,
        "volatility":volatility,
        "sharpe":sharpe,
        "annual_performance":df_annual_performance,
        "drawdown_max":drawdown_max,
        "drawdown_gfc":drawdown_gfc,
        "drawdown_pandemic":drawdown_pandemic,
        "drawdown_2022":drawdown_2022,
        "drawdown_summer_2024":drawdown_summer_2024,
        "spy_corrections":df_corrections,
        "data":df_ret,
    }

    return(result)

In [ ]:
res = backtester(frequency_rebalance="quarterly")

In [ ]:
res["annual_return"]

{'spy': np.float64(0.21061327318937573),
 'agg': np.float64(0.015644959126740243),
 'portfolio': np.float64(0.11061475222403394)}

In [ ]:
res["volatility"]

{'spy': np.float64(0.1547556156146824),
 'agg': np.float64(0.06990808383111777),
 'portfolio': np.float64(0.09418492426058639)}

In [ ]:
res["sharpe"]

{'spy': np.float64(1.312666228680625),
 'agg': np.float64(0.2569368132344841),
 'portfolio': np.float64(1.161088232042625)}

In [ ]:
res["drawdown_max"]

{'spy': np.float64(-0.16680317390187094),
 'agg': np.float64(-0.0978993660237314),
 'portfolio': np.float64(-0.12355884956574603)}

In [ ]:
res["data"]

,date,spy,agg,ret_spy,ret_agg,year,quarter,date_rebalance,before_rebal_spy,before_rebal_agg,portfolio_total_value,after_rebal_spy,after_rebal_agg,ret_portfolio,equity_spy,equity_agg,equity_portfolio,drawdown_spy,drawdown_agg,drawdown_portfolio
0,2022-06-30,363.724640,92.908958,0.000000,0.000000,2022,2,2022-06-30,0.500000,0.500000,1.000000,0.500000,0.500000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
1,2022-07-01,367.571564,93.683372,0.010576,0.008335,2022,3,2022-09-30,0.505288,0.504168,1.009456,0.505288,0.504168,0.009456,1.010576,1.008335,1.009456,0.000000,0.000000,0.000000
2,2022-07-05,368.265808,93.857300,0.001889,0.001857,2022,3,2022-09-30,0.506243,0.505104,1.011346,0.506243,0.505104,0.001873,1.012485,1.010207,1.011346,0.000000,0.000000,0.000000
3,2022-07-06,369.509552,93.225655,0.003377,-0.006730,2022,3,2022-09-30,0.507952,0.501704,1.009657,0.507952,0.501704,-0.001671,1.015905,1.003409,1.009657,0.000000,-0.006730,-0.001671
4,2022-07-07,375.043732,93.015121,0.014977,-0.002258,2022,3,2022-09-30,0.515560,0.500571,1.016131,0.515560,0.500571,0.006413,1.031120,1.001143,1.016131,0.000000,-0.008973,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,2024-12-24,601.299988,96.450081,0.011115,0.001138,2024,4,2024-12-31,0.685256,0.630597,1.315853,0.685256,0.630597,0.006309,1.653174,1.038114,1.315853,-0.007381,-0.039085,-0.014744
626,2024-12-26,601.340027,96.519844,0.000067,0.000723,2024,4,2024-12-31,0.685302,0.631053,1.316355,0.685302,0.631053,0.000381,1.653284,1.038865,1.316355,-0.007315,-0.038390,-0.014368
627,2024-12-27,595.010010,96.320511,-0.010527,-0.002065,2024,4,2024-12-31,0.678088,0.629750,1.307837,0.678088,0.629750,-0.006470,1.635880,1.036719,1.307837,-0.017764,-0.040376,-0.020745
628,2024-12-30,588.219971,96.699249,-0.011412,0.003932,2024,4,2024-12-31,0.670350,0.632226,1.302576,0.670350,0.632226,-0.004023,1.617212,1.040796,1.302576,-0.028973,-0.036603,-0.024685


## Looping Through Portfolios

In [ ]:
df_portfolios = pd.read_excel("..\\data\\portfolios_20250223.xlsx", "Sheet1")
df_portfolios

,name,start_date,end_date,rebal_frequency,spy,agg,hyg,tlt,gld,buffer_010,buffer_020,buffer_100,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,spy,2007-04-11,2024-12-31,daily,1.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
1,agg,2007-04-11,2024-12-31,daily,0.00,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
2,hyg,2007-04-11,2024-12-31,daily,0.00,0.0,1.0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
3,tlt,2007-04-11,2024-12-31,daily,0.00,0.0,0.0,1.00,0.00,0.00,0.00,0.00,0,0,0,0,0
4,gld,2007-04-11,2024-12-31,daily,0.00,0.0,0.0,0.00,1.00,0.00,0.00,0.00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,growth_model_no_duration,2022-06-30,2024-12-31,quarterly,0.65,0.0,0.0,0.15,0.10,0.05,0.00,0.05,0,0,0,0,0
926,growth model_3,2022-06-30,2024-12-31,quarterly,0.70,0.0,0.0,0.15,0.00,0.05,0.05,0.05,0,0,0,0,0
927,growth model_4,2022-06-30,2024-12-31,quarterly,0.70,0.0,0.0,0.15,0.05,0.00,0.05,0.05,0,0,0,0,0
928,sv_equity_buffer,2022-06-30,2024-12-31,quarterly,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0,0,0,1,0


In [ ]:
lst_name = []
lst_date_start = []
lst_date_end = []
lst_frequency_rebalance = []
lst_weight_spy = []
lst_weight_agg = []
lst_weight_hyg = []
lst_weight_tlt = []
lst_weight_gld = []
lst_weight_buffer_010 = []
lst_weight_buffer_020 = []
lst_weight_buffer_100 = []
lst_weight_sv_hedged_income = []
lst_weight_sv_hedged_balanced = []
lst_weight_sv_hedged_enhanced_growth = []
lst_weight_sv_equity_buffer = []
lst_weight_sv_equity_buffer_growth = []
lst_cumulative_ret = []
lst_annual_ret = []
lst_volatility = []
lst_sharpe = []
lst_annual_performance = []
lst_drawdown_max = []
lst_drawdown_gfc = []
lst_drawdown_pandemic = []
lst_drawdown_2022 = []
lst_drawdown_summer_2024 = []
for _ , row in df_portfolios.iterrows():
    name = row["name"]
    date_start = row["start_date"]
    date_end = row["end_date"]
    frequency_rebalance = row["rebal_frequency"]

    # generating portfolio
    portfolio = {}
    weight_spy = row["spy"]
    if weight_spy > 0:
        portfolio["spy"] = weight_spy
    
    weight_agg = row["agg"]
    if weight_agg > 0:
        portfolio["agg"] = weight_agg
        
    weight_hyg = row["hyg"]
    if weight_hyg > 0:
        portfolio["hyg"] = weight_hyg

    weight_tlt = row["tlt"]
    if weight_tlt > 0:
        portfolio["tlt"] = weight_tlt

    weight_gld = row["gld"]
    if weight_gld > 0:
        portfolio["gld"] = weight_gld
        
    weight_buffer_010 = row["buffer_010"]
    if weight_buffer_010 > 0:
        portfolio["buffer_010"] = weight_buffer_010
        
    weight_buffer_020 = row["buffer_020"]
    if weight_buffer_020 > 0:
        portfolio["buffer_020"] = weight_buffer_020
        
    weight_buffer_100 = row["buffer_100"]
    if weight_buffer_100 > 0:
        portfolio["buffer_100"] = weight_buffer_100

    weight_sv_hedged_income = row["sv_hedged_income"]
    if weight_sv_hedged_income > 0:
        portfolio["sv_hedged_income"] = weight_sv_hedged_income

    weight_sv_hedged_balanced = row["sv_hedged_balanced"]
    if weight_sv_hedged_balanced > 0:
        portfolio["sv_hedged_balanced"] = weight_sv_hedged_balanced

    weight_sv_hedged_enhanced_growth = row["sv_hedged_enhanced_growth"]
    if weight_sv_hedged_enhanced_growth > 0:
        portfolio["sv_hedged_enhanced_growth"] = weight_sv_hedged_enhanced_growth

    weight_sv_equity_buffer = row["sv_equity_buffer"]
    if weight_sv_equity_buffer > 0:
        portfolio["sv_equity_buffer"] = weight_sv_equity_buffer

    weight_sv_equity_buffer_growth = row["sv_equity_buffer_growth"]
    if weight_sv_equity_buffer_growth > 0:
        portfolio["sv_equity_buffer_growth"] = weight_sv_equity_buffer_growth

    # running backtest for strategy
    res = backtester(portfolio=portfolio, date_start=date_start, date_end=date_end, frequency_rebalance=frequency_rebalance, df_px=df_px, df_spy_corrections=df_spy_corrections)

    # creating annual performance DataFrame
    df_ap = res["annual_performance"].copy()
    df_ap["strategy"] = name
    df_ap["start"] = date_start
    df_ap["end"] = date_end
    df_ap["rebal_frequency"] = frequency_rebalance
    df_ap["spy"] = weight_spy
    df_ap["agg"] = weight_agg
    df_ap["hyg"] = weight_hyg
    df_ap["tlt"] = weight_tlt
    df_ap["gld"] = weight_gld
    df_ap["buffer_010"] = weight_buffer_010
    df_ap["buffer_020"] = weight_buffer_020
    df_ap["buffer_100"] = weight_buffer_100
    df_ap["sv_hedged_income"] = weight_sv_hedged_income
    df_ap["sv_hedged_balanced"] = weight_sv_hedged_balanced
    df_ap["sv_hedged_enhanced_growth"] = weight_sv_hedged_enhanced_growth
    df_ap["sv_equity_buffer"] = weight_sv_equity_buffer
    df_ap["sv_equity_buffer_growth"] = weight_sv_equity_buffer_growth
    lst_annual_performance.append(df_ap)
    
    # saving results to lists, which will be used to create a final results DataFrame
    lst_name.append(name)
    lst_date_start.append(date_start)
    lst_date_end.append(date_end)
    lst_frequency_rebalance.append(frequency_rebalance)
    lst_weight_spy.append(weight_spy)
    lst_weight_agg.append(weight_agg)
    lst_weight_hyg.append(weight_hyg)
    lst_weight_tlt.append(weight_tlt)
    lst_weight_gld.append(weight_gld)
    lst_weight_buffer_010.append(weight_buffer_010)
    lst_weight_buffer_020.append(weight_buffer_020)
    lst_weight_buffer_100.append(weight_buffer_100)
    lst_weight_sv_hedged_income.append(weight_sv_hedged_income)
    lst_weight_sv_hedged_balanced.append(weight_sv_hedged_balanced)
    lst_weight_sv_hedged_enhanced_growth.append(weight_sv_hedged_enhanced_growth)
    lst_weight_sv_equity_buffer.append(weight_sv_equity_buffer)
    lst_weight_sv_equity_buffer_growth.append(weight_sv_equity_buffer_growth)
    lst_cumulative_ret.append(res["cumulative_return"]["portfolio"])
    lst_annual_ret.append(res["annual_return"]["portfolio"])
    lst_volatility.append(res["volatility"]["portfolio"])
    lst_sharpe.append(res["sharpe"]["portfolio"])
    lst_drawdown_max.append(res["drawdown_max"]["portfolio"])
    lst_drawdown_gfc.append(res["drawdown_gfc"]["portfolio"])
    lst_drawdown_pandemic.append(res["drawdown_pandemic"]["portfolio"])
    lst_drawdown_2022.append(res["drawdown_2022"]["portfolio"])
    lst_drawdown_summer_2024.append(res["drawdown_summer_2024"]["portfolio"])
    
    print(portfolio, date_start, date_end, frequency_rebalance)

# creating results DataFrame
df_backtest_result = pd.DataFrame({
    "strategy":lst_name,
    "start":lst_date_start,
    "end":lst_date_end,
    "rebal_frequency":lst_frequency_rebalance,
    "spy":lst_weight_spy,
    "agg":lst_weight_agg,
    "hyg":lst_weight_hyg,
    "tlt":lst_weight_tlt,
    "gld":lst_weight_gld,
    "buffer_010":lst_weight_buffer_010,
    "buffer_020":lst_weight_buffer_020,
    "buffer_100":lst_weight_buffer_100,
    "sv_hedged_income":lst_weight_sv_hedged_income,
    "sv_hedged_balanced":lst_weight_sv_hedged_balanced,
    "sv_hedged_enhanced_growth":lst_weight_sv_hedged_enhanced_growth,
    "sv_equity_buffer":lst_weight_sv_equity_buffer,
    "sv_equity_buffer_growth":lst_weight_sv_equity_buffer_growth,
    "cumulative_ret":lst_cumulative_ret,
    "annual_ret":lst_annual_ret,
    "volatility":lst_volatility,
    "sharpe":lst_sharpe,
    "drawdown_max":lst_drawdown_max,
    "drawdown_gfc":lst_drawdown_gfc,
    "drawdown_pandemic":lst_drawdown_pandemic,
    "drawdown_2022":lst_drawdown_2022,
    "drawdown_summer_2024":lst_drawdown_summer_2024,
})
df_backtest_result

{'spy': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'agg': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'hyg': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'tlt': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'gld': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'buffer_010': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'buffer_020': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'buffer_100': 1.0} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.5, 'agg': 0.5} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.6, 'agg': 0.4} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.7, 'agg': 0.3} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.8, 'agg': 0.2} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.5, 'hyg': 0.5} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.6, 'hyg': 0.4} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.7, 'hyg': 0.3} 2007-04-11 00:00:00 2024-12-31 00:00:00 daily
{'spy': 0.

,strategy,start,end,rebal_frequency,spy,agg,hyg,tlt,gld,buffer_010,...,sv_equity_buffer_growth,cumulative_ret,annual_ret,volatility,sharpe,drawdown_max,drawdown_gfc,drawdown_pandemic,drawdown_2022,drawdown_summer_2024
0,spy,2007-04-11,2024-12-31,daily,1.00,0.0,0.0,0.00,0.00,0.00,...,0,4.698201,0.103270,0.198669,0.594232,-0.551894,-0.551894,-0.337173,-0.244964,-0.047200
1,agg,2007-04-11,2024-12-31,daily,0.00,1.0,0.0,0.00,0.00,0.00,...,0,0.653881,0.028822,0.054771,0.546328,-0.184329,-0.128353,-0.095792,-0.145087,-0.000797
2,hyg,2007-04-11,2024-12-31,daily,0.00,0.0,1.0,0.00,0.00,0.00,...,0,1.284977,0.047776,0.112376,0.471367,-0.342465,-0.341999,-0.220287,-0.155192,-0.009492
3,tlt,2007-04-11,2024-12-31,daily,0.00,0.0,0.0,1.00,0.00,0.00,...,0,0.701072,0.030459,0.154400,0.271482,-0.483512,-0.161931,-0.157277,-0.299897,0.000000
4,gld,2007-04-11,2024-12-31,daily,0.00,0.0,0.0,0.00,1.00,0.00,...,0,2.609571,0.075185,0.173417,0.504863,-0.455550,-0.294141,-0.125277,-0.210328,-0.014572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,growth_model_no_duration,2022-06-30,2024-12-31,quarterly,0.65,0.0,0.0,0.15,0.10,0.05,...,0,0.419110,0.150544,0.118066,1.246950,-0.142837,NaN,NaN,-0.140678,-0.027842
926,growth model_3,2022-06-30,2024-12-31,quarterly,0.70,0.0,0.0,0.15,0.00,0.05,...,0,0.421906,0.151451,0.123115,1.207157,-0.145769,NaN,NaN,-0.145496,-0.029750
927,growth model_4,2022-06-30,2024-12-31,quarterly,0.70,0.0,0.0,0.15,0.05,0.00,...,0,0.424904,0.152423,0.121887,1.225011,-0.145759,NaN,NaN,-0.144549,-0.029074
928,sv_equity_buffer,2022-06-30,2024-12-31,quarterly,0.00,0.0,0.0,0.00,0.00,0.00,...,0,0.302513,0.111695,0.065159,1.657901,-0.069329,NaN,NaN,-0.069329,-0.019957


In [ ]:
df_annual_performance = pd.concat(lst_annual_performance)
df_annual_performance

,year,ret_portfolio,strategy,start,end,rebal_frequency,spy,agg,hyg,tlt,gld,buffer_010,buffer_020,buffer_100,sv_hedged_income,sv_hedged_balanced,sv_hedged_enhanced_growth,sv_equity_buffer,sv_equity_buffer_growth
0,2007,0.029855,spy,2007-04-11,2024-12-31,daily,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,2008,-0.367950,spy,2007-04-11,2024-12-31,daily,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
2,2009,0.263518,spy,2007-04-11,2024-12-31,daily,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3,2010,0.150562,spy,2007-04-11,2024-12-31,daily,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4,2011,0.018950,spy,2007-04-11,2024-12-31,daily,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2023,0.140715,sv_equity_buffer,2022-06-30,2024-12-31,quarterly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
2,2024,0.121645,sv_equity_buffer,2022-06-30,2024-12-31,quarterly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
0,2022,0.022959,sv_equity_buffer_growth,2022-06-30,2024-12-31,quarterly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
1,2023,0.177437,sv_equity_buffer_growth,2022-06-30,2024-12-31,quarterly,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1


## Exporting to Excel

In [ ]:
with pd.ExcelWriter("backtest.xlsx", engine="openpyxl", mode="w", datetime_format="D/M/YYYY") as writer: 
    df_backtest_result.to_excel(writer, sheet_name="results", index=False, startrow=0, startcol=0)
    df_annual_performance.to_excel(writer, sheet_name="annual_performance", index=False, startrow=0, startcol=0)